<a href="https://colab.research.google.com/github/17prateek12/customer-behavior-predict-recommendation/blob/main/Predict_%26recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler


In [42]:
url='https://raw.githubusercontent.com/17prateek12/customer-behavior-predict-recommendation/main/shopping_trends_updated.csv'

In [43]:
df=pd.read_csv(url)

In [44]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [45]:
df['Frequency of Purchases']=df['Frequency of Purchases'].str.lower()

In [46]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['Gender']= label_encoder.fit_transform(df['Gender'])
df['Category']= label_encoder.fit_transform(df['Category'])
df['Location']= label_encoder.fit_transform(df['Location'])
df['Size']= label_encoder.fit_transform(df['Size'])
df['Color']= label_encoder.fit_transform(df['Color'])
df['Season']= label_encoder.fit_transform(df['Season'])
df['Subscription Status']= label_encoder.fit_transform(df['Subscription Status'])
df['Shipping Type']= label_encoder.fit_transform(df['Shipping Type'])
df['Discount Applied']= label_encoder.fit_transform(df['Discount Applied'])
df['Promo Code Used']= label_encoder.fit_transform(df['Promo Code Used'])
df['Payment Method']= label_encoder.fit_transform(df['Payment Method'])
df['Frequency of Purchases']= label_encoder.fit_transform(df['Frequency of Purchases'])

In [47]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,1,Blouse,1,53,16,0,7,3,3.1,1,1,1,1,14,5,3
1,2,19,1,Sweater,1,64,18,0,12,3,3.1,1,1,1,1,2,1,3
2,3,50,1,Jeans,1,73,20,2,12,1,3.1,1,2,1,1,23,2,6
3,4,21,1,Sandals,2,90,38,1,12,1,3.5,1,3,1,1,49,4,6
4,5,45,1,Blouse,1,49,36,1,21,1,2.7,1,2,1,1,31,4,0


In [48]:
X = df.drop(['Frequency of Purchases','Discount Applied','Subscription Status','Promo Code Used','Item Purchased'], axis=1)
y = df['Frequency of Purchases']

In [49]:
X.head()

,Customer ID,Age,Gender,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Shipping Type,Previous Purchases,Payment Method
0,1,55,1,1,53,16,0,7,3,3.1,1,14,5
1,2,19,1,1,64,18,0,12,3,3.1,1,2,1
2,3,50,1,1,73,20,2,12,1,3.1,2,23,2
3,4,21,1,2,90,38,1,12,1,3.5,3,49,4
4,5,45,1,1,49,36,1,21,1,2.7,2,31,4


In [50]:
X_train, X_test, y_train_frequency, y_test_frequency = train_test_split(
    X, y, test_size=0.35, random_state=42
)

In [51]:
kmeans = KMeans(n_clusters=len(df['Frequency of Purchases'].unique()))
kmeans.fit(X_train,y_train_frequency)
kmeans_labels_train = kmeans.labels_
frequency_accuracy = accuracy_score(y_train_frequency, kmeans_labels_train)
print(f"K-Means: ", frequency_accuracy )


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


K-Means:  0.14792899408284024


In [52]:
svm_model = SVC(kernel='rbf', C=100000.0,degree = 2)
svm_model.fit(X_train, y_train_frequency)
svm_pred_test = svm_model.predict(X_test)
frequency_accuracy = accuracy_score(y_test_frequency, svm_pred_test)
print("SVM: ",frequency_accuracy)


SVM:  0.15384615384615385


In [53]:
knn_model = KNeighborsClassifier(n_neighbors=40, weights="uniform",algorithm="auto")
knn_model.fit(X_train, y_train_frequency)
knn_pred_test = knn_model.predict(X_test)
frequency_accuracy = accuracy_score(y_test_frequency, knn_pred_test)
print("KNN: ",frequency_accuracy)


KNN:  0.16263736263736264


In [54]:
rf_classifier_frequency = RandomForestClassifier(n_estimators=110, max_depth=15, random_state=42)
rf_classifier_frequency.fit(X_train, y_train_frequency)
frequency_predictions = rf_classifier_frequency.predict(X_test)
frequency_accuracy = accuracy_score(y_test_frequency, frequency_predictions)
print("Random Forest:", frequency_accuracy)


Random Forest: 0.1523809523809524


In [55]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train_frequency)
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test_frequency, y_pred)
print("Xgboost :", accuracy)

Xgboost : 0.1443223443223443


In [56]:
X_train_sequence = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_sequence = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [57]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_sequence.shape[1], X_train_sequence.shape[2]), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(df['Frequency of Purchases'].unique()), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [58]:
model.fit(X_train_sequence, y_train_frequency, validation_data=(X_test_sequence, y_test_frequency),epochs=20,batch_size=32,verbose=1)

Epoch 1/20
80/80 [==============================] - 2s 9ms/step - loss: 15.7579 - accuracy: 0.1302 - val_loss: 4.0378 - val_accuracy: 0.1348
Epoch 2/20
80/80 [==============================] - 0s 6ms/step - loss: 2.4449 - accuracy: 0.1550 - val_loss: 2.2043 - val_accuracy: 0.1260
Epoch 3/20
80/80 [==============================] - 0s 6ms/step - loss: 2.2018 - accuracy: 0.1582 - val_loss: 2.0735 - val_accuracy: 0.1311
Epoch 4/20
80/80 [==============================] - 0s 5ms/step - loss: 2.0852 - accuracy: 0.1471 - val_loss: 2.1272 - val_accuracy: 0.1502
Epoch 5/20
80/80 [==============================] - 0s 6ms/step - loss: 2.0598 - accuracy: 0.1511 - val_loss: 2.0592 - val_accuracy: 0.1304
Epoch 6/20
80/80 [==============================] - 0s 6ms/step - loss: 2.0689 - accuracy: 0.1325 - val_loss: 2.1756 - val_accuracy: 0.1414
Epoch 7/20
80/80 [==============================] - 1s 7ms/step - loss: 2.0486 - accuracy: 0.1416 - val_loss: 2.1723 - val_accuracy: 0.1245
Epoch 8/20
80/80 [=

In [59]:
_, accuracy = model.evaluate(X_test_sequence, y_test_frequency)
print(f"LSTM Accuracy: {accuracy}")

43/43 [==============================] - 0s 2ms/step - loss: 2.0329 - accuracy: 0.1487
LSTM Accuracy: 0.1487179547548294


In [60]:
def get_user_recommendations(Location,predicted_frequency):
    user_dff = df[(df['Frequency of Purchases'] == predicted_frequency) & (df['Location'] == Location)]
    user_dff['Item Purchased'] = (user_dff['Item Purchased'])
    return user_dff[['Item Purchased']]


Location=list(X_test['Location'])[1]
predicted_frequency= knn_model.predict(X_test)[1]
recommendations = get_user_recommendations(Location,predicted_frequency)
print("Recommendations for the User:")
print(recommendations)


Recommendations for the User:
     Item Purchased
173             Hat
262           Pants
278         Sweater
443      Sunglasses
511           Shirt
1772          Shoes
2350          Shoes
2638          Shirt


<ipython-input-60-732cfb00f8f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_dff['Item Purchased'] = (user_dff['Item Purchased'])
